In [1]:
%load_ext autoreload
%autoreload 2

from stock import Stock
from stock import Webpage
from stock import url2html
from scrape import search

import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import xlsxwriter

In [2]:
maxis = Stock("maxis-bhd")

Stock Cd: maxis-bhd
Stock Id: 41688
Growth Rate: 0.08
Beta: 0.49
Discount Rate: 0.09


In [8]:
maxis.balance_sheet

,0,1,2,3,4
0,Total Current Assets,2986,2667.66,2470.2,2457.88
2,Cash and Short Term Investments,582,560.46,602.13,682.35
3,Cash,137,190.18,162.76,197.1
4,Cash & Equivalents,582,560,602,-
5,Short Term Investments,445,370.27,439.37,485.24
6,"Total Receivables, Net",1961,1711.22,1510.83,1295.42
7,"Accounts Receivables - Trade, Net",1349,1111.88,1104.94,976.66
8,Total Inventory,3,15.92,4.49,5.94
9,Prepaid Expenses,157,159.33,156.34,210.44
10,"Other Current Assets, Total",283,220.73,196.41,263.73


In [20]:
url = "https://simplywall.st/stocks/my/transportation/klse-airasia/airasia-group-berhad-shares"
html = url2html(url)
soup = BeautifulSoup(html)

In [60]:
m = re.search(r"fair value \(MYR([+-]?\d+\.\d+)\)", soup.text)
m.groups()[0]

'1.75'

In [12]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0',
}

params = (
    ('q', 'airasia'),
)

response = requests.get('https://www.investing.com/search/', headers=headers, params=params)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('https://www.investing.com/search/?q=airasia', headers=headers, cookies=cookies)


In [16]:
soup = BeautifulSoup(response.text)

In [19]:
result = soup.find('a', ['js-inner-all-results-quote-item'])
result

<a class="js-inner-all-results-quote-item row" href="/equities/airasia-bhd">
<span class="flag first"><i class="ceFlags middle Malaysia"></i></span>
<span class="second">AIRA</span>
<span class="third">Airasia Bhd</span>
<span class="fourth">Stock - Kuala Lumpur  equities</span>
</a>

In [22]:
print(result['href'])

/equities/airasia-bhd


In [8]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0',
}

data = {
  'search_text': 'hong leong'
}

response = requests.post('https://www.investing.com/search/service/searchTopBar', headers=headers, data=data)
response.text

'    <!DOCTYPE HTML>\n    <html dir="ltr" xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" xmlns:schema="http://schema.org/" class="com" lang="en" geo="MY">\n    <head>\n    <script src="https://cookie-cdn.cookiepro.com/scripttemplates/otSDKStub.js"  type="text/javascript" charset="UTF-8" data-domain-script="2c339abd-8f8b-4350-ac5f-b942aed18814"></script>\n<script type="text/javascript">\n    function OptanonWrapper() { }\n\n    // temp for checks\n    </script>\n<style>\n\t.ot-floating-button {\n\t\tz-index:12 !important;\n\t}\n</style>\n\n    <script src="https://i-invdn-com.akamaized.net/js/jquery-6.4.9.04.min.js"></script>\n    <script>\n        (function(i,s,o,g,r,a,m){i[\'GoogleAnalyticsObject\']=r;i[r]=i[r]||function(){\n            (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),\n            m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n        })(window,document,\'script\',\'//www.google-analytics.com/analytic

In [11]:
response.json()

JSONDecodeError: Expecting value: line 1 column 5 (char 4)

In [30]:
total_assets = search("Total Assets", maxis.balance_sheet)
total_liabilities = search("Total Liabilities", maxis.balance_sheet)
current_shares_outstanding = search("Total Common Shares Outstanding", maxis.balance_sheet)
total_equity = search("Total Equity", maxis.balance_sheet)

net_assets = total_assets - total_liabilities
net_assets

7070.0

In [31]:
net_asset_value = net_assets / current_shares_outstanding
net_asset_value = round(net_asset_value, 2)
net_asset_value

0.9

In [ ]:
# Overview
Last Price
52 wk Range
EPS
Volume
Market Cap
Dividend (Yield)
P/E Ratio
Next Earnings Date

In [ ]:
EPS ()
(Earnings/Latest No. Of Shares)
NAV ()
(Shareholders' Equity/Latest No. Of Shares)
 How to Calculate Shareholders' Equity. Shareholders' equity may be calculated by subtracting its total liabilities from its total assets—both of which are itemized on a company's balance sheet. Total assets can be categorized as either current or non-current assets.
Price Earnings Ratio (PER)
(Price/EPS)
Net Earnings Margin
(Net Earnings/Revenue)
Revenue Growth
((Current Year Revenue - Last Year Revenue) / Last Year Revenue)
Net Earnings Growth
((Current Year Earnings - Last Year Earnings) / Last Year Earnings)
Return On Asset (ROA)
(Net Earnings/Total Assets)
Return On Equity (ROE)
(Net Earnings/Equity)
Current Ratio
(Current Assets/Current Liabilities)

In [8]:
df =  pd.DataFrame(columns=['', 'Company', 'Industry'])
df.append(maxis.ratios.head(6), ignore)

,,Company,Industry,0,1,2
0,NaN,NaN,NaN,P/E Ratio TTM,29.27,22.17
1,NaN,NaN,NaN,Price to Sales TTM,4.42,3.74
2,NaN,NaN,NaN,Price to Cash Flow MRQ,93.21,101.87
3,NaN,NaN,NaN,Price to Free Cash Flow TTM,78.68,134.26
4,NaN,NaN,NaN,Price to Book MRQ,5.86,18.44
5,NaN,NaN,NaN,Price to Tangible Book MRQ,-,294.09


In [ ]:
PE
price to book
dividend
price cash
price sales

In [21]:
float('-9')

-9.0

In [22]:
search("EPS\(MRQ\) vs Qtr. 1 Yr. Ago MRQ", maxis.ratios)

-0.1351

In [32]:
table4 = {
    "EPS": search("Basic EPS ANN", maxis.ratios),
    "EPS(MRQ) vs Qtr. 1 Yr. Ago MRQ": search("EPS\(MRQ\) vs Qtr. 1 Yr. Ago MRQ", maxis.ratios),
    "EPS(TTM) vs TTM 1 Yr. Ago TTM": search("EPS\(TTM\) vs TTM 1 Yr. Ago TTM", maxis.ratios),
    "5 Year EPS Growth 5YA": search("5 Year EPS Growth 5YA", maxis.ratios),
        
    "Return on Equity TTM": search("Return on Equity TTM", maxis.ratios),
    "Return on Equity 5YA": search("Return on Equity 5YA", maxis.ratios),
        
    "Price to Earnings Ratio": search("P/E Ratio TTM", maxis.ratios),
        
    "Dividend per Share": search("Dividend Yield ANN", maxis.ratios),
    "Dividend Yield 5 Year Avg. 5YA": search("Dividend Yield 5 Year Avg. 5YA", maxis.ratios),
    "Dividend Growth Rate ANN": search("Dividend Growth Rate ANN", maxis.ratios),
        
    "Net Asset per Share": net_asset_value,
    "Price to Book": search("Price to Book MRQ", maxis.ratios),
    "LT Debt to Equity": search("LT Debt to Equity", maxis.ratios),
}

In [33]:
len(table4)

13

In [ ]:
# Required data, table 1
worksheet.write_column('A1', table4.keys())
worksheet.write_column('B1', table4.values(), colored_format)

# rewrite in percentage
worksheet.write('B5', maxis.growth_rate()[0], percentage_format)
worksheet.write('B7', maxis.discount_rate()[0], percentage_format)

## Analyse stock

In [285]:
excel_name = maxis.stock_cd + ".xlsx"
sheet_name = "Sheet1"

# writer = pd.ExcelWriter(excel_name, engine='xlsxwriter') writer.save()
workbook = xlsxwriter.Workbook(excel_name)
worksheet = workbook.add_worksheet(sheet_name)

worksheet.set_column('A:A', 20)
worksheet.set_column('A:I', 10)

currency_format = workbook.add_format({'num_format': '$#,##0.00'})
percentage_format = workbook.add_format({
    'num_format': '0.0%',
    'bg_color': '#dae8ec',
    'border': 1})

colored_format = workbook.add_format({
    'num_format': '$#,##0.00',
    'bg_color': '#dae8ec',
    'border': 1
})

In [142]:
# Stock 
table1 = {
    "Name of Stock": maxis.stock_cd.replace("-", " ").title(),
    "Operating Cash Flow": search("Cash From Operating Activities", maxis.cash_flow()),
    "Total Debt": search("Total Long Term Debt", maxis.balance_sheet()),
    "Cash & Equivalent": search("Cash & Equivalent", maxis.balance_sheet()),
    "Growth Rate": 0,
    "No. of Shares Outstanding": search("Shares Outstanding", maxis.overview())/1000000,
    "Discount Rate": 0
}

/home/xunwei/anaconda3/envs/stock/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


0

In [265]:
# Required data, table 1
worksheet.write_column('A1', table1.keys())
worksheet.write_column('B1', table1.values(), colored_format)

# rewrite in percentage
worksheet.write('B5', maxis.growth_rate()[0], percentage_format)
worksheet.write('B7', maxis.discount_rate()[0], percentage_format)

/home/xunwei/anaconda3/envs/stock/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


0

In [246]:
# Ten-year cash flow calculations, bottom table
start_row = 11

# headers
worksheet.write_column(start_row, 0, ["Year", "Cash Flow", "Discount Rate", "Discounted Value"])
worksheet.write_row(start_row, 1, list(range(now.year, now.year + 10, 1)))

# calculations
cash_flow = ["=B2*(1+B5)"]
cash_flow.extend(["=" + chr(ord('B') + i) + str(start_row+2) + "*(1+$B$5)" for i in range(10)])
# +1, +2
cf_row = start_row + 1
for i in range(10):
    worksheet.write_formula(cf_row, i+1, cash_flow[i], currency_format)
    
# +2, +3
dr_row = start_row + 2
discount_rate = ["=1/(1 + $B$7)^" + str(i) for i in range(1, 11)]
for i in range(10):
    worksheet.write_formula(dr_row, i+1, discount_rate[i])

# +3, +4
dv_row = start_row + 3
discounted_value = ["=PRODUCT("+chr(ord('B')+i)+str(cf_row+1)+":"+chr(ord('B')+i)+str(dr_row+1)+")" for i in range(10)]
for i in range(10):
    worksheet.write_formula(dv_row, i+1, discounted_value[i], currency_format)

In [247]:
# Intrinsic values, table 2
worksheet.write_column('D2', ["PV of 10 yr Cash Flows", "Intrinsic Value per Share", 
                              "- Debt per Share", "+ Cash per share", "net Cash per Share"])
worksheet.write_column('E2', [f"=SUM(B{dv_row+1}:K{dv_row+1})", "=E2/B6", "=B3/B6", "=B4/B6", "=E3-E4+E5"], colored_format)

0

In [248]:
# Stock overview, table 3
df = maxis.overview().reset_index(drop=True)
index = [0, 5, 6, 7, 8, 9, 11, 15]
worksheet.write_column('G2', df.iloc[index, 0])
worksheet.write_column('H2', df.iloc[index, 1])

0

In [249]:
# report
worksheet.write_column('A18', [maxis.growth_rate()[1], maxis.beta()[1]])

/home/xunwei/anaconda3/envs/stock/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


0

In [42]:
workbook.close()
# Shift + Ctrl + F9

## Rename excel

In [280]:
import os
import numpy as np

In [273]:
operating_cf = search("Cash From Operating Activities", maxis.cash_flow())
shares_outstanding = search("Shares Outstanding", maxis.overview())/1000000

In [278]:
last_price = search("Last Price", maxis.overview())

In [267]:
cash_flow = []
for i in range(1, 11):
    operating_cf = operating_cf * (1 + maxis.growth_rate)
    cash_flow.append(operating_cf)

In [282]:
values = cash_flow
rate = maxis.discount_rate

npv = (values / (1+rate)**np.arange(1, len(values)+1)).sum(axis=0) / shares_outstanding

'''
#            rate            values      
a =  np.npv(maxis.discount_rate, cash_flow) 
print("Net Present Value(npv) : ", a) 

(values / (1+rate)**np.arange(0, len(values))).sum(axis=0)
'''

'\n#            rate            values      \na =  np.npv(maxis.discount_rate, cash_flow) \nprint("Net Present Value(npv) : ", a) \n\n(values / (1+rate)**np.arange(0, len(values))).sum(axis=0)\n'

In [289]:
os.rename(excel_name, maxis.stock_cd + "-" + str(round(npv, 2)) + "-" + str(last_price) + ".xlsx") 

In [290]:
!jupyter nbconvert --to script Scrape_from_investing.ipynb

[NbConvertApp] Converting notebook Scrape_from_investing.ipynb to script
[NbConvertApp] Writing 11719 bytes to Scrape_from_investing.py
